In [5]:
!pip install --upgrade pip
!pip install python-decouple
!pip install geoalchemy2
!pip install shapely
!pip install scipy
!pip install keras
!pip install tensorflow

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (19.1.1)
     |████████████████████████████████| 92.5MB 62.8MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 33.8MB/s eta 0:00:01
     |████████████████████████████████| 3.2MB 40.4MB/s eta 0:00:01
     |████████████████████████████████| 368kB 55.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 42.8MB/s ta 0:00:01
     |████████████████████████████████| 92kB 38.0MB/s eta 0:00:01
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ee/98/38/46cbcc5a93cfea5492d19c38562691ddb23b940176c14f7b48
Successfully built gast termcolor absl-py


In [6]:
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker
from decouple import config
from shapely import wkb, wkt
from shapely.geometry import Point
from geoalchemy2.shape import to_shape 

import pandas as pd
import numpy as np
import random
import json
from datetime import datetime, timedelta
import re
from matplotlib import pyplot as plt

from sklearn.preprocessing import RobustScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import GridSearchCV


# ----------- TODO: Issues with importing imbalance-learn library
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import NearMiss
# from imblearn.under_sampling import (RandomUnderSampler,
#                                      ClusterCentroids,
#                                      TomekLinks,
#                                      NeighbourhoodCleaningRule,
#                                      NearMiss)

Using TensorFlow backend.


In [7]:
"""Contains models for DB."""

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, BigInteger, Integer, String, DateTime, ForeignKey, Float
from sqlalchemy.orm import relationship
from geoalchemy2 import Geometry


BASE = declarative_base()


class City(BASE):
    """City model for DB. Has information of cities."""
    __tablename__ = 'city'
    id            = Column(BigInteger, primary_key=True)
    city          = Column(String, unique=False, nullable=False)
    state         = Column(String, unique=False, nullable=True)
    country       = Column(String, unique=False, nullable=False)
    location      = Column(Geometry(geometry_type='POINT'), nullable=False)
    blocks        = relationship("Blocks", back_populates="city")
    zipcodes      = relationship("ZipcodeGeom", back_populates="city")
    incidents     = relationship("Incident", back_populates="city")


class Blocks(BASE):
    """Block model for DB. Has information of city blocks for a related city
        id."""
    __tablename__ = 'block'
    id            = Column(BigInteger, primary_key=True)
    cityid        = Column(BigInteger, ForeignKey('city.id'), nullable=False)
    shape         = Column(Geometry(geometry_type='MULTIPOLYGON'), nullable=False)
    population    = Column(Integer, nullable=False)
    city          = relationship("City", back_populates="blocks")
    incidents     = relationship("Incident", back_populates="block")

class ZipcodeGeom(BASE):
    """Zipcode geometry model for DB. Has information of zipcodes and related
        city id."""
    __tablename__ = 'zipcodegeom'
    id            = Column(BigInteger, primary_key=True)
    cityid        = Column(BigInteger, ForeignKey('city.id'), nullable=False)
    zipcode       = Column(String, nullable=False, unique=True)
    shape         = Column(Geometry(geometry_type='MULTIPOLYGON'), nullable=False)
    city          = relationship("City", back_populates="zipcodes")

class Incident(BASE):
    """Incident model for DB. Has information of a specific crime, including
        where it took place, when it took place, and the type of crime that
        occurred."""
    __tablename__ = 'incident'
    id            = Column(BigInteger, primary_key=True)
    crimetypeid   = Column(BigInteger, ForeignKey('crimetype.id'), nullable=False)
    locdescid     = Column(BigInteger, ForeignKey('locdesctype.id'), nullable=False)
    cityid        = Column(BigInteger, ForeignKey('city.id'), nullable=False)
    blockid       = Column(BigInteger, ForeignKey('block.id'), nullable=False)
    location      = Column(Geometry(geometry_type='POINT'), nullable=False)
    datetime      = Column(DateTime, nullable=False)
    hour          = Column(Integer, nullable=False)
    dow           = Column(Integer, nullable=False)
    month         = Column(Integer, nullable=False)
    year          = Column(Integer, nullable=False)
    city          = relationship("City", back_populates="incidents")
    block         = relationship("Blocks", back_populates="incidents")
    crimetype     = relationship("CrimeType", back_populates="incidents")
    locationdesc  = relationship("LocationDescriptionType", back_populates="incidents")

class CrimeType(BASE):
    """CrimeType model for DB. Has information of the types of crime, including
        a general description and the numerical severity of the crime."""
    __tablename__ = 'crimetype'
    id            = Column(BigInteger, primary_key=True)
    category      = Column(String, unique=True, nullable=False)
    severity      = Column(Integer, nullable=False)
    incidents     = relationship("Incident", back_populates="crimetype")


class LocationDescriptionType(BASE):
    """Location description model for DB. Has information on the type of
        location that the crime took place."""
    __tablename__ = 'locdesctype'
    id            = Column(BigInteger, primary_key=True)
    key1          = Column(String, nullable=False)
    key2          = Column(String, nullable=False)
    key3          = Column(String, nullable=False)
    incidents     = relationship("Incident", back_populates="locationdesc")

In [8]:
class GetData(object):
    def go(self, SESSION, start_year, end_year):
        SQL_QUERY = \
            f'''
                SELECT
                    incident.blockid,
                    incident.year,
                    incident.month,
                    incident.dow,
                    incident.hour,
                    SUM(crimetype.severity)/AVG(block.population) AS severity
                FROM incident
                INNER JOIN block ON incident.blockid = block.id
                INNER JOIN crimetype ON incident.crimetypeid = crimetype.id
                    AND block.population > 0
                    AND incident.cityid = 1
                    AND incident.year >= {start_year}
                    AND incident.year <= {end_year}
                GROUP BY
                    incident.blockid,
                    incident.year,
                    incident.month,
                    incident.dow,
                    incident.hour
            '''
        return SESSION.execute(text(SQL_QUERY)).fetchall()

In [22]:
def process_data(data, start_year, end_year, map_risk):

    NUM_BLOCKIDS = 801
    
    X = np.zeros((NUM_BLOCKIDS, 24, 7*24))
    y = np.zeros((NUM_BLOCKIDS, 12, 7*24))
    blockid_dict = {}

    # Create random array (BLOCKIDS) from 1-801 inclusive
    # of length NUM_BLOCKIDS
    BLOCKIDS = random.choices(list(range(1,802)), k=NUM_BLOCKIDS)
    
    for ind, blockid in enumerate(BLOCKIDS):
        blockid_dict[blockid] = ind

    # records is the list of rows we get from the query with this order:
    #   blockid, year, month, dow, hour, risk
    #   month is from 1 - 12

    for r in data:
        if r[0] in blockid_dict:
            if r[1] == end_year:
                # index into array  0-based month
                # vvvvvvvvvvvvvvvv    vvvvvv
                y[blockid_dict[r[0]], r[2]-1, 24*r[3]+r[4]] = map_risk(float(r[5]))
                #                             ^^^^^^^^^^^^^   ^^^^
                #                             hours since     risk
                #                             beginning of
                #                             week
            else:
                # index into array    year 0.....1   month   
                # vvvvvvvvvvvvvvvv    vvvvvvvvvvvvv  vvvvvv
                X[blockid_dict[r[0]], 12*(r[1]-start_year)+r[2]-1, 24*r[3]+r[4]] = map_risk(float(r[5]))
                #                                                  ^^^^^^^^^^^^^   ^^^^
                #                                                  hours since     risk
                #                                                  beginning of
                #                                                  week                
    
    return X, y

In [23]:
from contextlib import contextmanager

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""

    DB_URI  = config('DB_URI')
    ENGINE  = create_engine(DB_URI)
    Session = sessionmaker(bind=ENGINE)
    SESSION = Session()
    
    try:
        yield SESSION
        SESSION.commit()
    except:
        SESSION.rollback()
        raise
    finally:
        SESSION.close()


def ready_data(training_start_year, training_end_year,
               testing_start_year, testing_end_year,
               map_risk):
    with session_scope() as session:
        training_data = GetData().go(session,
                                     training_start_year,
                                     training_end_year)
        testing_data = GetData().go(session,
                                     testing_start_year,
                                     testing_end_year)
        
        X_train, y_train = process_data(training_data,
                                        training_start_year, 
                                        training_end_year,
                                        map_risk)
        X_test, y_test = process_data(testing_data,
                                      testing_start_year, 
                                      testing_end_year,
                                      map_risk)

    return X_train, X_test, y_train, y_test

In [24]:
epochs = 5
def map_risk_1(risk):
    return np.power(risk * 1e5, 2)

def map_risk_0(risk):
    return risk

def map_risk_2(risk):
    return np.power(risk * 1e5, 3)

def map_risk_3(risk):
    return np.power(risk * 1e5, 4)

map_risk = [map_risk_1, map_risk_2, map_risk_3]  # , map_risk_1, map_risk_2, map_risk_3]

In [25]:
%%time
X_train, X_test, y_train, y_test = ready_data(2015, 2017, 2016, 2018, map_risk_0)
print(len(X_train), len(y_train), len(X_test), len(y_test))

801 801 801 801
CPU times: user 2.9 s, sys: 343 ms, total: 3.24 s
Wall time: 9.83 s


In [26]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape 

((801, 24, 168), (801, 12, 168), (801, 24, 168), (801, 12, 168))

In [27]:
X_train_saved, y_train_saved, X_test_saved, y_test_saved = X_train, y_train, X_test, y_test

In [14]:
X_train = pd.DataFrame(data=X_train, columns=['risk_past'])
y_train = pd.DataFrame(data=y_train, columns=['risk_future'])
X_test = pd.DataFrame(data=X_test, columns=['risk_past'])
y_test = pd.DataFrame(data=y_test, columns=['risk_future'])

In [39]:
def plot_output(y, y_pred, dataset_type):

    fig = plt.figure(figsize=(10, 8))
    plt.plot(np.arange(len(y.flatten())),
                 y.flatten(), color='blue');
    plt.plot(np.arange(len(y_pred.flatten())),
                 y_pred.flatten(), color='red');
    plt.xlabel('Hour since beginning of data', fontsize=16)
    plt.ylabel('Risk', fontsize=18)
    plt.title(dataset_type + ' dataset', fontsize=18)
    plt.legend(labels=['risk', 'predicted risk'], prop={'size': 20})
    plt.show()

## LSTM Predictions

In [28]:
def plot_training_validation_loss(epochs, history):
    plt.plot(range(1, epochs), history.history['loss'])
    plt.plot(range(1, epochs), history.history['val_loss'])
    plt.title('model train vs validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.show()

In [29]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
import keras.backend as K
from keras.wrappers.scikit_learn import KerasRegressor

def perc_error(y_true, y_pred):
    return 100.0 * K.mean((y_true - y_pred) / y_true)

def create_model(learn_rate=0.01, 
                 momentum=0,
                 opt_name='Adam',
                 init_mode='uniform',
                 activation='relu'
                ):
    data_dim    = 7 * 24   # All values in each hour of the week
                           # averaged over each day for all weeks
                           # of the month
    timesteps   = 2 * 12   # Summed per month
    batch_size  = 64
    num_outputs = 7 * 24 * 12
    
    # expected input data shape: (batch_size, timesteps, data_dim)
    model = Sequential()
    model.add(LSTM(1024, input_shape=(24,168), activation='relu'))
    model.add(Dense(12*168, kernel_initializer='zero', activation='relu'))
    model.add(Reshape((12,168)))
    
    optimizer = { 
        'Adam':     Adam(lr=learn_rate),
    }

    model.compile(loss='mean_squared_error', optimizer=optimizer[opt_name])

    return model

def predict(X_train, y_train, X_test, y_test):
    
    # GridSearchCV's fit method requires X which is 2D, y which is 1D.
    # This is a problem for us since our X and y are 3D.
    # Instead of GridSearchCV, we will create our own loop to
    # search through the grid.
    # 
    batch_size = 64
    for epochs in range(5, 6, 5):
        for lr in [0.0001, 0.001, 0.01, 0.1]:
            for opt_name in ['Adam']:
                for init_mode in ['zero']:
                    for activation in ['relu']:

                        print('>'*80)
                        print('epochs:', epochs, '   lr:', lr, \
                              '   opt:', opt_name, \
                              '   init:', init_mode, \
                              '   act:', 'relu')
                        print('>'*80)

                        model = create_model()

                        model.compile(loss='mean_squared_error', optimizer='adam')

                        history = model.fit(X_train, y_train,
                                            batch_size=batch_size, 
                                            epochs=epochs,
                                            validation_data=(X_test, y_test))

                        mse = model.evaluate(X_test, y_test,
                                             batch_size=batch_size)
                        print('<'*80)
                        print('epochs:', epochs, '   lr:', lr, \
                              '   opt:', opt_name, \
                              '   init:', init_mode, \
                              '   act:', 'relu', \
                              ' Test MSE:', mse \
                        )
                        print('<'*80)

In [30]:
predict(X_train, y_train, X_test, y_test)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epochs: 5    lr: 0.0001    opt: Adam    init: zero    act: relu
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Instructions for updating:
Use tf.cast instead.
Train on 801 samples, validate on 801 samples
Epoch 1/5
801/801 [==============================] - 6s 7ms/step - loss: 2.6612e-06 - val_loss: 8.0647e-07
Epoch 2/5
801/801 [==============================] - 4s 6ms/step - loss: 2.6612e-06 - val_loss: 8.0647e-07
Epoch 3/5
801/801 [==============================] - 4s 6ms/step - loss: 2.6612e-06 - val_loss: 8.0647e-07
Epoch 4/5
801/801 [==============================] - 4s 6ms/step - loss: 2.6612e-06 - val_loss: 8.0647e-07
Epoch 5/5
801/801 [==============================] - 1s 1ms/step
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
epochs: 5    lr: 0.0001    opt: Adam    init: zero    act: relu  Test MSE: 8.064734240409446e-07
<<<<<<<<<

In [31]:
y_train_pred = model.predict(X_train).flatten()
y_test_pred = model.predict(X_test).flatten()

plot_output(y_train, y_train_pred, 'Training')
plot_output(y_test, y_test_pred, 'Testing')

NameError: name 'model' is not defined

In [ ]:
def print_y_stats(s, y):
    num_spaces = 2
    if isinstance(y, pd.core.series.Series):
        y_flat = y
    elif isinstance(y, np.ndarray):
        y_flat = y.flatten()
    else:
        raise ValueError('Could not process type:', type(y))
        
    print(s)
    print(' ' * num_spaces, 'min: ', min(y_flat))
    print(' ' * num_spaces, 'max: ', max(y_flat))
    print(' ' * num_spaces, 'mean:', np.mean(y_flat))
    print(' ' * num_spaces, 'std: ', np.std(y_flat))
    return min(y_flat), max(y_flat), np.mean(y_flat), np.std(y_flat)

In [ ]:
_, _, _, y_train_std = print_y_stats('y_train:', y_train)
print()
print_y_stats('y_test:', y_test)

## Remove outliers

### Let's try filtering values that are 20 standard deviations above the mean

In [ ]:
std = np.std(y_train)
y_train_filtered = y_train[y_train < 20*std] # Remove all values larger than 20 standard deviations

std = np.std(y_test)
y_test_filtered = y_test[y_test < 20*std]   # Remove all values larger than 20 standard deviations

print('Number of values filtered from y_train:', len(y_train[y_train > 20*std]))
print('Number of values filtered from y_test:', len(y_test[y_test > 20*std]))
fig = plt.figure(figsize=(12, 8))
plt.plot(y_train_filtered, color='blue');
plt.plot(y_test_filtered, color='red');
plt.legend(labels=['training set data', 'testing set data'], prop={'size': 20})
plt.show()

### With our threshold of 20 * std, we have removed 13 points from y_train and 12 from y_test. This is out of 1.6 million points, so they were defintely outliers. Let's run the prediction again with the updated y values

In [ ]:
y_train_updated = np.where(y_train.flatten() < 20*std, y_train.flatten(), [0.]*len(y_train.flatten()))
y_test_updated  = np.where(y_test.flatten() < 20*std, y_test.flatten(), [0.]*len(y_test.flatten()))

predict(X_train, 
        y_train_updated.reshape((801, 12, 168)), 
        X_test, 
        y_test_updated.reshape((801, 12, 168)))

In [ ]:
y_train_updated_pred = model.predict(X_train).flatten()
y_test_updated_pred = model.predict(X_test).flatten()

plot_output(y_train_updated, y_train_updated_pred, 'Training')
plot_output(y_test_updated, y_test_updated_pred, 'Testing')

### Distribution of y-values

In [ ]:
fig = plt.figure(figsize=(12,8))
pd.Series(y_train_updated).hist(bins=np.arange(0.0001, 0.002, 0.0002));

In [ ]:
fig = plt.figure(figsize=(12,8))
pd.Series(y_test_updated).hist(bins=np.arange(0.0001, 0.002, 0.0002));

### This data seems reasonable, although it is a little lopsided. Still, it shouldn't cause the neural network to give us the large error that it is giving. Let's try giving the network the full set of features.

## Use the full set of features

In [ ]:
class GetDataFull(object):
    def go(self, SESSION, start_year, end_year):
        SQL_QUERY = \
            f'''
            SELECT incident.blockid, 
                    incident.year, 
                    incident.month, 
                    incident.dow, 
                    incident.hour,
                    SUM(crimetype.severity)/AVG(block.population) AS severity
            FROM incident
            INNER JOIN block ON incident.blockid = block.id INNER JOIN crimetype ON incident.crimetypeid = crimetype.id AND block.population > 0
                AND block.population > 0
                AND severity > 0
                AND incident.cityid = 1
                AND incident.year >= {start_year}
                AND incident.year <= {end_year}
            GROUP BY
                incident.blockid,
                incident.year,
                incident.month,
                incident.dow,
                incident.hour
            '''
        return SESSION.execute(text(SQL_QUERY)).fetchall()

In [ ]:
def days_in_month(year, month):
    p = pd.Period(f'{year}-{month}-1')
    return p.days_in_month

def day_of_week(dt):
    return dt.weekday()

def create_arrays(blockids, start_year, end_year):
    idx = 0
    X_blockid, X_year, X_month, X_dow, X_hour, X_risk = [], [], [], [], [], []
    for blockid in blockids:
        for year in range(start_year, end_year + 1):
            for month in range(1, 12 + 1):      # month range is 1-12
                for day in range(1, days_in_month(year, month) + 1):
                    for hour in range(24):      # hour range is 0-23
                        X_blockid.append(blockid)
                        X_year.append(year)
                        X_month.append(month)
                        X_dow.append(day_of_week(datetime(year, month, day)))
                        X_hour.append(hour)
                        X_risk.append(0.0)
                        idx += 1
    
    X = pd.DataFrame({'blockid':  X_blockid,
                      'year':     X_year,
                      'month':    X_month,
                      'dow':      X_dow,
                      'hour':     X_hour,
                      'risk':     X_risk})

    return X

In [ ]:
def process_data_full(data, start_year, end_year):

    def remove_outliers_from_risk(risk):
        std = np.std(risk)
        risk = np.where(risk < 20*std, 
                     risk, 
                     [0.]*len(risk)).reshape(risk.shape)

        return risk
    
    NUM_BLOCKIDS = 801
    
    delta_years = end_year - start_year + 1
    
    blockid_dict = {}

    # Create random array (BLOCKIDS) from 1-801 inclusive
    # of length NUM_BLOCKIDS
    BLOCKIDS = random.choices(list(range(1,802)), k=NUM_BLOCKIDS)
    
    for ind, blockid in enumerate(BLOCKIDS):
        blockid_dict[blockid] = ind

    blockids = list(blockid_dict.values())
    X = create_arrays(blockids, start_year, end_year)

    # records is the list of rows we get from the query with this order:
    #   blockid, year, month, dow, hour, risk
    #   month is from 1 - 12

    X1 = []
    for r in data:
        if r[0] in blockid_dict:            
            X1.append((r[0], r[1], r[2], r[3], r[4], r[5]))

    X1 = pd.DataFrame(data=X1,
                      columns=['blockid', 'year', 'month', 'dow', 'hour','risk2'])
    X = pd.merge(X, X1, 
                 how='left',
                 left_on=['blockid', 'year', 'month', 'dow', 'hour'],
                 right_on=['blockid', 'year', 'month', 'dow', 'hour']
                )
    X['all_risk'] = X.risk.astype(float) + X.risk2.astype(float)
    X = X.drop(columns=['risk', 'risk2']) \
         .rename(mapper={'all_risk': 'risk'}, axis=1)
    
    y = X['risk'].copy()
    X = X.drop(columns=['risk']).copy()
    y = remove_outliers_from_risk(y)
    
    return X, y

In [ ]:
def ready_data_full(training_start_year, training_end_year,
                    testing_start_year, testing_end_year):
    with session_scope() as session:
        training_data = GetDataFull().go(session,
                                         training_start_year,
                                         training_end_year)
        testing_data = GetDataFull().go(session,
                                         testing_start_year,
                                         testing_end_year)
        X_train, y_train = process_data_full(training_data,
                                             training_start_year, 
                                             training_end_year)
        X_test, y_test = process_data_full(testing_data,
                                           testing_start_year, 
                                           testing_end_year)

    return X_train, X_test, y_train, y_test

In [ ]:
%%time
X_train, X_test, y_train, y_test = ready_data_full(2015, 2016, 2017, 2018)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# def predict_full(X_train, y_train, X_test, y_test):
#     data_dim    = 5 # 7 * 24   # All values in each hour of the week
#     timesteps   = 2 * 12   # Summed per month
#     batch_size  = 64
#     num_outputs = 7 * 24 * 12

#     # expected input data shape: (batch_size, timesteps, data_dim)
#     model = Sequential()
#     model.add(LSTM(1024, return_sequences=True, 
# #                    input_shape=(timesteps, # 24
# #                                 data_dim), # 168
#                    activation='relu',
#                    kernel_initializer='random_uniform',
#                    bias_initializer='zeros'
#                   )
#              )
#     model.add(LSTM(1024, return_sequences=True, input_shape=(timesteps, data_dim)))
#     model.add(LSTM(128, input_shape=(timesteps, data_dim), activation='relu'))
#     model.add(Dense(num_outputs, activation='relu'))
#     model.add(Reshape((12, 7 * 24)))

#     model.compile(loss='mean_squared_error',
#                    optimizer=Adam(lr=0.1))

#     history = model.fit(X_train, y_train,
#                         batch_size=batch_size, epochs=10,
#                         validation_data=(X_test, y_test))

#     mse = model.evaluate(X_test, y_test,
#                          batch_size=batch_size)
#     print('Test MSE:', mse)
#     return history, model

In [ ]:
# history, model = predict_full(X_train, y_train, X_test, y_test)